In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/ai-cup-2025-fall

/content/drive/.shortcut-targets-by-id/10S0Z48fYfB80KAsAP5OAwLfjbbfT-jiw/ai-cup-2025-fall


In [ ]:
!ls

 12_best.pt		       output
 42_testing_image.zip	       runs
 42_training_image.zip	       Test1.pt
 42_training_label.zip	       test2
 945_best.pt		      'Test2-for-Second-Split (1).pt'
 ai-cup-test1.ipynb	       Test2-for-Second-Split.pt
 ai-cup-test2.ipynb	       test3_for_1200_epoch_start.pt
 analysis.csv		       test_data
 analysis_raw.csv	       test_for_freeze_10_765.pt
 analysis-test2.csv	       train.csv
 dataset		       training_image
 dataset_alternating	       training_label
 dataset_alternating.zip       train.txt
 data.yaml		       val.csv
 detectioin.ipynb	       val.txt
 history		       val_vis
'labels_stat (1).csv'	       yolo11n.pt
 labels_stat.csv	       yolo12l.pt
 labels_stat_split.csv	       yolo12m.pt
 laiyuhung.ipynb	       yolo12n.pt
 mAP0.566_20250920_022830.pt   yolo12s.pt
 mAP0.655_20250920_070850.pt   yolo12x.pt


In [ ]:
!pip install ultralytics

In [ ]:
import os
import pandas as pd
import shutil
from datetime import datetime

def find_best_by_median(base_dir="runs/detect", output_dir="runs/detect"):
    best_overall = {"exp": None, "median": 0, "csv": None}
    results = []

    for root, dirs, files in os.walk(base_dir):
        if "results.csv" in files:
            exp_name = os.path.basename(root)  # train, train1, train2 ...
            csv_path = os.path.join(root, "results.csv")

            try:
                df = pd.read_csv(csv_path)
                # mAP50-95 的中位數
                median_map5095 = df["metrics/mAP50-95(B)"].median()
                results.append((exp_name, median_map5095))

                if median_map5095 > best_overall["median"]:
                    best_overall.update({
                        "exp": exp_name,
                        "median": median_map5095,
                        "csv": csv_path
                    })
            except Exception as e:
                print(f"⚠️ 無法讀取 {csv_path}: {e}")

    # 列出所有實驗的 median
    print("📊 各實驗 mAP50-95 中位數：")
    for exp_name, median in results:
        print(f" - {exp_name}: median mAP50-95={median:.3f}")

    # 找到最佳實驗
    if best_overall["exp"]:
        best_weights = os.path.join(base_dir, best_overall["exp"], "weights", "best.pt")
        if os.path.exists(best_weights):
            # 產生檔名
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            new_name = f"mAP{best_overall['median']:.3f}_{timestamp}.pt"
            dest_path = os.path.join(output_dir, new_name)

            shutil.copy(best_weights, dest_path)
            print(f"\n🏆 最佳模型：{best_overall['exp']}")
            print(f"   median mAP50-95={best_overall['median']:.3f}")
            print(f"   已複製到: {dest_path}")
        else:
            print(f"⚠️ {best_weights} 不存在！")
    else:
        print("❌ 沒找到任何 results.csv")


# 執行
find_best_by_median("runs/detect", "runs/detect")


📊 各實驗 mAP50-95 中位數：
 - R01_normal_20251003_145215: median mAP50-95=0.660
 - R01_hard_20251003_145215: median mAP50-95=0.549
 - R02_normal_20251003_145215: median mAP50-95=0.661
 - R02_hard_20251003_145215: median mAP50-95=0.567
 - R03_normal_20251003_145215: median mAP50-95=0.675
 - R03_hard_20251003_145215: median mAP50-95=0.556
 - R04_normal_20251003_145215: median mAP50-95=0.684
 - R04_hard_20251003_145215: median mAP50-95=0.570
 - R05_normal_20251003_145215: median mAP50-95=0.683
 - R05_hard_20251003_145215: median mAP50-95=0.578
 - R06_normal_20251003_145215: median mAP50-95=0.684
 - R01_normal_20251004_122846: median mAP50-95=0.517
 - R01_hard_20251004_122846: median mAP50-95=0.425
 - R02_normal_20251004_122846: median mAP50-95=0.580
 - R02_hard_20251004_122846: median mAP50-95=0.470
 - R03_normal_20251004_122846: median mAP50-95=0.605
 - R03_hard_20251004_122846: median mAP50-95=0.488
 - R04_normal_20251004_122846: median mAP50-95=0.637
 - R04_hard_20251004_122846: median mAP50-

In [ ]:
import os
import random
import pandas as pd
import cv2
from torch.utils.data import Dataset, DataLoader
from ultralytics import YOLO
import albumentations as A

# =========================
# Step 1. 載入資料
# =========================
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")

# 自動生成 train.txt / val.txt
with open("train.txt", "w") as f:
    for path in train_df["image_path"].tolist():
        f.write(path + "\n")

with open("val.txt", "w") as f:
    for path in val_df["image_path"].tolist():
        f.write(path + "\n")

# 自動生成 data.yaml (YOLO 格式資料夾)
yaml_content = f"""
train: dataset/images/train
val: dataset/images/val

nc: 1
names: ["target"]
"""
with open("data.yaml", "w") as f:
    f.write(yaml_content.strip())

print("✅ 已生成 data.yaml，指向 dataset/images/")

# =========================
# Step 2. 定義 Augmentation (只針對正樣本)
# =========================
augment_transform = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.HorizontalFlip(p=0.5),
])

# =========================
# Step 3. Dataset
# =========================
class YOLOCustomDataset(Dataset):
    def __init__(self, samples, augment=False):
        self.samples = samples
        self.augment = augment

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        img = cv2.imread(sample["image_path"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        labels = []
        if os.path.exists(sample["label_path"]):
            with open(sample["label_path"], "r") as f:
                labels = f.read().splitlines()

        # 正樣本 + augment=True → 做增強
        if self.augment and sample.get("has_label", False) and len(labels) > 0:
            img = augment_transform(image=img)["image"]

        return img, labels

# =========================
# Step 4. DataLoader (≥40% 正樣本)
# =========================
batch_size = 16
min_pos_ratio = 0.4
min_pos = int(batch_size * min_pos_ratio)

pos_samples = train_df[train_df["has_label"] == True].to_dict("records")
neg_samples = train_df[train_df["has_label"] == False].to_dict("records")

train_dataset = YOLOCustomDataset(train_df.to_dict("records"), augment=True)
val_dataset = YOLOCustomDataset(val_df.to_dict("records"), augment=False)

def collate_fn(_):
    pos_batch = random.sample(pos_samples, min_pos)
    neg_batch = random.sample(neg_samples, batch_size - min_pos)
    batch_samples = pos_batch + neg_batch
    random.shuffle(batch_samples)
    return batch_samples

train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

print(f"✅ DataLoader 已建立，每個 batch 至少 {int(min_pos_ratio*100)}% 正樣本")



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ 已生成 data.yaml，指向 dataset/images/
✅ DataLoader 已建立，每個 batch 至少 40% 正樣本


In [ ]:
# !cp -r "/content/drive/MyDrive/ai-cup-2025-fall/dataset" "/content/dataset"


In [ ]:
# !ls runs/detect/mAP0.715_20250921_071859.pt

In [ ]:
import os
import shutil
from ultralytics import YOLO

# =========================
# Step 1. 搬移資料到本機 /content/dataset
# =========================
gdrive_path = "/content/drive/MyDrive/ai-cup-2025-fall/dataset"
local_path = "/content/dataset"

if os.path.exists(local_path):
    shutil.rmtree(local_path)

print("📂 正在複製資料集到本機 ...")
shutil.copytree(gdrive_path, local_path)
print("✅ 完成搬移到:", local_path)

📂 正在複製資料集到本機 ...
✅ 完成搬移到: /content/dataset


In [ ]:
!pip install -U ultralytics


In [ ]:
yaml_content = f"""
train: {local_path}/images/train
val: {local_path}/images/val

nc: 1
names: ["target"]
"""
with open("data.yaml", "w") as f:
    f.write(yaml_content.strip())

print("✅ 已生成 data.yaml，指向本機 /content/dataset")

✅ 已生成 data.yaml，指向本機 /content/dataset


In [ ]:
from ultralytics import YOLO

# ================================
# 1️⃣ 載入上次最佳權重
# ================================
model = YOLO("test_for_freeze_10_765.pt")

# ================================
# 2️⃣ 全層 Fine-Tune (High-Precision)
# ================================

model.train(
    data="data.yaml",
    epochs=200,
    batch=32,
    imgsz=960,
    device=0,
    workers=8,
    amp=True,

    # === ✅ 關鍵修正 ===
    optimizer="AdamW",        # 更穩定
    lr0=0.0003,               # 降低起始 LR（避免震盪）
    lrf=0.01,                 # 收斂時 LR 比例
    momentum=0.9,
    weight_decay=0.0005,
    freeze=0,                 # 全層 fine-tune
    warmup_epochs=5,          # 延長 warmup
    warmup_momentum=0.8,
    patience=40,              # 提早停止前多觀察一點

    # Loss 權重
    box=9.0,                 # 稍微降低，避免 loss 不穩
    cls=3.0,
    dfl=6.0,
    dropout=0.05,

    # Augmentation（降低隨機性）
    mosaic=0.1,
    mixup=0.0,
    copy_paste=0.01,
    erasing=0.02,
    translate=0.05,
    scale=0.9,
    degrees=2,
    fliplr=0.5,
    flipud=0.2,

    # Logging
    project="/content/drive/MyDrive/ai-cup-2025-fall/runs/detect",
    name="test_for_freezeAfter",
    exist_ok=True,
)


Ultralytics 8.3.209 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=9.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=3.0, compile=False, conf=None, copy_paste=0.01, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=2, deterministic=True, device=0, dfl=6.0, dnn=False, dropout=0.05, dynamic=False, embed=None, epochs=200, erasing=0.02, exist_ok=True, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=test_for_freeze_10_765.pt, momentum=0.9, mosaic=0.1, multi_scale=False, name=test_for_freezeAfter, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=40, perspective=0.0, plots=True,

KeyboardInterrupt: 

### PHASE 1

In [ ]:


# # =========================
# # Step 2. 載入模型
# # =========================
# model = YOLO("runs/detect/mAP0.759_20251007_074926.pt")

# # =========================
# # Step 3. 開始訓練 (純YOLO官方增強)
# # =========================
# model.train(
#     data="data.yaml",
#     epochs=500,
#     batch=16,
#     imgsz=896,
#     workers=8,
#     device=0,
#     amp=True,

#     # -----------------
#     # 📦 Loss 權重
#     # -----------------
#     box=10,    # 強化定位準確度
#     cls=2.0,    # 難分類樣本強化分類權重
#     dfl=3,      # 穩定分佈焦點損失

#     # -----------------
#     # 🩻 醫學CT專用 Augmentation
#     # -----------------
#     degrees=5,               # 小角度旋轉，模擬切層誤差
#     translate=0.08,          # 輕微平移
#     scale=0.55,              # 縮放比例變化（模擬不同 FOV）
#     shear=1.5,               # 輕仿射
#     perspective=0.0008,      # 輕透視偏移

#     fliplr=0.5,              # 水平翻轉
#     flipud=0.15,             # 垂直翻轉（少量使用）

#     mosaic=0.3,              # 控制性混合影像
#     mixup=0.1,               # 小比例樣本融合
#     copy_paste=0.05,         # 輕度貼合增加樣本
#     erasing=0.15,            # 遮擋模擬（類似 cutout）

#     # -----------------
#     # 🌗 灰階亮度對比模擬（安全區間）
#     # -----------------
#     hsv_h=0.0,               # 關閉色相
#     hsv_s=0.0,               # 關閉飽和度
#     hsv_v=0.25,              # 僅保留亮度變化

#     # -----------------
#     # 📉 學習率策略
#     # -----------------
#     cos_lr=True,
#     patience=30,
# )

# print("🚀 醫學CT專用強化訓練開始！（無外掛增強）")

### PHASE 2

In [ ]:
# # =====================================================
# # YOLOv12 Dual-Phase Fine-Tuning Script for Aortic CT
# # Author: Yi-Chin Chen x ChatGPT (2025.10)
# # =====================================================

# from ultralytics import YOLO
# import shutil, os

# # ===============================
# # Step 1. 載入前一階段最佳模型
# # ===============================
# pretrained_model = "runs/detect/mAP0.759_20251009_023032.pt"   # ← 換成你的 pretrain 路徑
# model = YOLO(pretrained_model)

# # ===============================
# # Step 2. Phase 2.5 - 冻結 backbone 穩定微調
# # ===============================
# print("\n🧩 Phase 2.5: Freeze Backbone Fine-Tuning 開始...\n")

# model.train(
#     data="data.yaml",
#     epochs=60,                  # 初期穩定階段
#     imgsz=896,
#     batch=16,
#     device=0,
#     workers=8,
#     amp=True,

#     # ---- Fine-tune 策略 ----
#     lr0=0.0005,
#     lrf=1e-5,
#     optimizer="AdamW",
#     freeze=10,                  # 凍結前 10 層 backbone
#     patience=40,
#     cos_lr=True,

#     # ---- Loss 權重 ----
#     box=12.0,
#     cls=2.5,
#     dfl=3.5,

#     # ---- CT 專用增強 ----
#     degrees=3,
#     translate=0.05,
#     scale=0.6,
#     shear=1.0,
#     perspective=0.0006,
#     fliplr=0.5,
#     flipud=0.1,
#     mosaic=0.1,
#     mixup=0.05,
#     copy_paste=0.02,
#     erasing=0.1,
#     hsv_h=0.0, hsv_s=0.0, hsv_v=0.15,
#     rect=True,                  # 維持病灶比例
#     dropout=0.03,
#     project="runs/detect",
#     name="Phase2.5_freeze",
#     exist_ok=True
# )

# # 取得 Phase 2.5 最佳權重路徑
# best_phase25 = "runs/detect/Phase2.5_freeze/weights/best.pt"
# if not os.path.exists(best_phase25):
#     raise FileNotFoundError("Phase 2.5 權重未生成，請確認前段訓練是否成功")

# # ===============================
# # Step 3. Phase 3 - 全層解凍高 IoU 精修
# # ===============================
# print("\n🚀 Phase 3: Full-Unfreeze High-IoU Fine-Tuning 開始...\n")

# model = YOLO(best_phase25)

# model.train(
#     data="data.yaml",
#     epochs=120,                 # 第二階段精修
#     imgsz=896,
#     batch=8,                    # 適度縮小以穩定梯度
#     device=0,
#     workers=8,
#     amp=True,

#     # ---- 高 IoU 訓練策略 ----
#     lr0=0.001,                  # 提高初始學習率以跳出局部最優
#     lrf=1e-5,
#     optimizer="AdamW",
#     freeze=0,                   # 全層解凍
#     patience=80,
#     cos_lr=True,
#     warmup_epochs=3,
#     warmup_momentum=0.8,
#     dropout=0.05,
#     rect=True,

#     # ---- Loss 強化 ----
#     box=8.0,
#     cls=2.0,
#     dfl=6.0,                    # 提高 DFL 權重 → 高 IoU 敏感

#     # ---- 進階增強 ----
#     degrees=4,
#     translate=0.08,
#     scale=0.65,
#     shear=1.2,
#     perspective=0.001,
#     fliplr=0.5,
#     flipud=0.15,
#     mosaic=0.25,
#     mixup=0.1,
#     copy_paste=0.05,
#     erasing=0.15,
#     hsv_h=0.0, hsv_s=0.0, hsv_v=0.25,
#     auto_augment="ta_wide",     # 更強對比變化
#     project="runs/detect",
#     name="Phase3_unfreeze",
#     exist_ok=True
# )

# # ===============================
# # Step 4. 驗證與結果輸出
# # ===============================
# print("\n📈 全流程完成，開始最終驗證...\n")

# best_phase3 = "runs/detect/Phase3_unfreeze/weights/best.pt"
# if os.path.exists(best_phase3):
#     metrics = model.val(data="data.yaml", model=best_phase3, imgsz=896, iou=0.5)
#     print("\n✅ 最終模型驗證完成：")
#     print(metrics)
# else:
#     print("⚠️ Phase 3 權重未找到，請檢查訓練輸出資料夾。")

# print("\n🎯 全階段微調流程完成！模型將聚焦於高 IoU 主動脈邊界辨識。")


In [ ]:
from ultralytics import YOLO

# 載入你剛才複製出來的模型
model = YOLO("runs/detect/mAP0.771_20251010_032824.pt")

# 在驗證集上做評估
metrics = model.val(
    data="data.yaml",   # 這裡要放你的 data.yaml (有定義 train/val 路徑)
    split="val",        # 指定用 validation set
    imgsz=640,          # 評估影像大小 (跟訓練一致)
    batch=16,           # 可以調整 batch size
    device=0            # GPU
)

print(metrics)  # 會包含 mAP50, mAP50-95, precision, recall


In [ ]:
# import os
# import cv2
# import torch
# from ultralytics import YOLO

# # =========================
# # Step 1. 準備路徑
# # =========================
# val_img_dir = "dataset/images/val"
# val_label_dir = "dataset/labels/val"
# save_dir = "val_vis"
# os.makedirs(save_dir, exist_ok=True)

# # =========================
# # Step 2. 載入模型
# # =========================
# model = YOLO("runs/detect/mAP0.759_20251007_062515.pt")  # 換成你的 best.pt

# # =========================
# # Step 3. IoU 計算
# # =========================
# def compute_iou(box1, box2):
#     """ box = [x1, y1, x2, y2] """
#     xi1 = max(box1[0], box2[0])
#     yi1 = max(box1[1], box2[1])
#     xi2 = min(box1[2], box2[2])
#     yi2 = min(box1[3], box2[3])
#     inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

#     box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
#     box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

#     union_area = box1_area + box2_area - inter_area
#     if union_area == 0:
#         return 0.0
#     return inter_area / union_area

# # =========================
# # Step 4. 驗證集推論 + 畫圖
# # =========================
# for img_file in os.listdir(val_img_dir):
#     if not img_file.endswith((".jpg", ".png", ".jpeg")):
#         continue

#     img_path = os.path.join(val_img_dir, img_file)
#     label_path = os.path.join(val_label_dir, img_file.replace(".jpg", ".txt").replace(".png", ".txt"))

#     # 讀圖
#     img = cv2.imread(img_path)
#     h, w = img.shape[:2]

#     # 標註框 (YOLO txt 格式: class cx cy w h)
#     gt_boxes = []
#     if os.path.exists(label_path):
#         with open(label_path, "r") as f:
#             for line in f.readlines():
#                 cls, cx, cy, bw, bh = map(float, line.strip().split())
#                 x1 = int((cx - bw / 2) * w)
#                 y1 = int((cy - bh / 2) * h)
#                 x2 = int((cx + bw / 2) * w)
#                 y2 = int((cy + bh / 2) * h)
#                 gt_boxes.append([x1, y1, x2, y2])
#                 cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 綠色 = GT

#     # 模型推論
#     results = model(img_path, conf=0.25, iou=0.5, verbose=False)

#     for r in results:
#         for box in r.boxes:
#             x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
#             conf = float(box.conf[0])

#             # 計算 IoU (取 max IoU 與任一 GT)
#             iou_val = 0
#             if gt_boxes:
#                 iou_val = max([compute_iou([x1, y1, x2, y2], g) for g in gt_boxes])

#             # 畫預測框 (紅色)
#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
#             cv2.putText(img, f"conf={conf:.2f} IoU={iou_val:.2f}",
#                         (x1, max(15, y1 - 5)), cv2.FONT_HERSHEY_SIMPLEX,
#                         0.5, (0, 0, 255), 1)

#     # 存圖
#     save_path = os.path.join(save_dir, img_file)
#     cv2.imwrite(save_path, img)

# print(f"✅ 已完成！結果存到 {save_dir}")
